In [37]:
import pinecone
import pypdf
from langchain.document_loaders import PyPDFDirectoryLoader #load the directory with PDFs
from langchain.text_splitter import RecursiveCharacterTextSplitter #to split the text into chunks
from langchain_openai.embeddings import OpenAIEmbeddings  #to create embeddings
from langchain.vectorstores import  Pinecone  as PVS #to store vectors in pinecone changed name due to namespace collision with pinecone
from langchain_openai import OpenAI  #openAI model
import os


In [29]:
from dotenv import load_dotenv
load_dotenv()  #load environment variables

True

In [19]:
def read_docs(dictionary):  #read all files in dictionary
    load_documents=PyPDFDirectoryLoader(dictionary)
    docs=load_documents.load()
    return docs

In [20]:
docs=read_docs("documents/")
len(docs)             

1

In [21]:
def chunk_data(docs, chunk_size, chunk_overlap):  #divide it into chunks
    text_spitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs=text_spitter.split_documents(docs)
    return docs

In [22]:
docs=chunk_data(docs=docs, chunk_size=800, chunk_overlap=50)
len(docs)

5

In [23]:
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY']) #create embeddings
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x00000248E88B4FD0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x00000248E85A9490>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [24]:
vectors=embeddings.embed_query('Hello how are you')
len(vectors)

1536

In [31]:
Pinecode_API_Key=os.environ['PINECONE_API_KEY']
Pinecode_API_Key

'1314520f-f997-413e-86c3-37f633e34edb'

In [33]:
#connecting to Pinecone instance
from pinecone import Pinecone
pc = Pinecone(api_key=Pinecode_API_Key)
from pinecone import ServerlessSpec
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)
index_name = 'langchain'

In [38]:
index=PVS.from_documents(docs, embeddings,index_name=index_name)

In [39]:
def retrive_query(query, k=2):
    matching_results=index.similarity_search(query, k=k)
    return matching_results

In [43]:
#create the chain and setup the LLm model
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI
llm=OpenAI(temperature=0.5)
chain=load_qa_chain(llm, chain_type='stuff')

In [41]:
def retrieve_ans(query):
    doc_search=retrive_query(query)  #get the matching vectors
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)  #search the ans to the query in those vectors
    return response


In [44]:
our_query = "who is the policy owner?"
answer = retrieve_ans(our_query)
print(answer)

[Document(page_content='PREMIUM RECEIPT\nTax Invoice\nReceipt No.: 1068910686 Receipt Date : 11-NOV-22\nPersonal Details\nUnique Invoice Number\nApplication No.: 6122268480 Policy No.: 0530974760\nPolicy Holder’s Name: SANDHYA DAMODAR SAWANT PAN No.: HAGPS4294M\nE-mail: sandhyadsawant6@gmail.com Contact No.: 8208364449\nPolicy Details\nPlan Name: BAJAJ ALLIANZ LIFE ETOUCH\nLife Insured: MRS SANDHYA DAMODAR SAWANT\nSum Assured: 10000000 Policy Commencement Date: 11-OCT-22\nPolicy Term: 40 Premium Payment Term: 40\nPremium (W/O GST): 624 Premium Payment Frequency: 12\nPremium Due Date From: 11-NOV-22 Premium Due Date To:\nPremium Received (incl GST): 736 Next Due Date: 11-DEC-22\nAmount in Words: Rupees Seven Hundred Thirty Six Only\nAgent Code: 3WA0000009 Agent Name: POLICYBAZAAR INSURANCE BROKERS PVT LTD\nTax Details', metadata={'page': 0.0, 'source': 'documents\\Premium Receipts.pdf'}), Document(page_content='reinstatement of policy, insurance cover shall resume from the date of reins